In [37]:
%pylab inline
import numpy as np

num_egs = 5
num_features = 1
num_classifiers = 3

X = np.zeros((num_egs,num_features))
labels = np.zeros(num_egs)
X[0][0]=10; labels[0] = 1;
X[1][0]=30; labels[1] = 1;
X[2][0]=40; labels[2] = -1;
X[3][0]=60; labels[3] = -1;
X[4][0]=90; labels[4] = 1;

# X[0][0]=10; labels[0] = 1;
# X[1][0]=20; labels[1] = -1;
# X[2][0]=30; labels[2] = 1;


Populating the interactive namespace from numpy and matplotlib


/home/praateek/miniconda3/envs/twitter/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['indices', 'sign']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [38]:
D = np.ones(num_egs) / num_egs
best_alphas = np.zeros(num_classifiers)
best_indexes = np.zeros(num_classifiers)
best_splits = np.zeros(num_classifiers)
best_signs = ["" for i in range(num_classifiers)]

In [39]:
def find_error(X, y, indices, weights, feature_index, feature_value):
    loss = 0
    for index in indices:
        sign = "lt"
        if X[index][feature_index] <= feature_value:
            if y[index] != -1:
                loss += D[index]
        else:
            if y[index] != 1:
                loss += D[index]
    if loss > 0.5:
        sign = "gt"
        loss = 1 - loss
    return loss, sign

def update_D(X, y, weight, alpha, feature_index, feature_value, best_sign):
    # Normalization constant
    Z = 0
    for i in range(len(X)):
        if best_sign == "lt":
            if X[i][feature_index] <= feature_value:
                weight[i] *= np.exp(alpha * y[i])
                Z += weight[i]
            else:
                weight[i] *= np.exp(-alpha * y[i])
                Z += weight[i]
        elif best_sign == "gt":
            if X[i][feature_index] >= feature_value:
                weight[i] *= np.exp(alpha * y[i])
                Z += weight[i]
            else:
                weight[i] *= np.exp(-alpha * y[i])
                Z += weight[i]
            
    weight = weight/Z
    return weight
def predict(X, alphas, best_indices, best_splits, best_signs):
    predicted = np.zeros(X.shape[0])
    for num, eg in enumerate(X):
        a = 0
        for i in range(len(alphas)):
            if best_signs[i] == "lt":
                if eg[int(best_indices[i])] <= best_splits[i]:
                    a += alphas[i] * -1
                else:
                    a += alphas[i]
            elif best_signs[i] == 'gt':
                if eg[int(best_indices[i])] >= best_splits[i]:
                    a += alphas[i] * -1
                else:
                    a += alphas[i]
        predicted[num] = np.sign(a)
    return predicted

In [40]:
for t in range(num_classifiers):
    best_error = float('inf')
    for feature in range(num_features):
        indices = np.argsort( X[:,feature] );
        X_temp = X[indices]
        # TODO add unique values here
        for eg in range(num_egs-1):
            mid_point = (X_temp[eg, feature] + X_temp[eg+1, feature])/2
            loss, sign = find_error(X, labels, indices, D, feature, mid_point)
            if best_error > loss:
                best_error = loss
                best_feature = feature
                best_value = mid_point
                best_sign = sign
                
    alpha = np.log((1 - best_error)/best_error) * 0.5
    
    best_alphas[t] = alpha
    best_indexes[t] = best_feature
    best_splits[t] = best_value
    best_signs[t] = best_sign
    
    D = update_D(X, labels, D, alpha, best_feature, best_value, best_sign)
    
    print("t = {} : threshold : {} : feature : {} : loss {}  : alpha = {} : sign : {} : D : ".format(t,best_value, best_feature, best_error,alpha, best_sign), D)



t = 0 : threshold : 35.0 : feature : 0 : loss 0.19999999999999996  : alpha = 0.6931471805599454 : sign : gt : D :  [0.125 0.125 0.125 0.125 0.5  ]
t = 1 : threshold : 75.0 : feature : 0 : loss 0.24999999999999997  : alpha = 0.5493061443340549 : sign : lt : D :  [0.25       0.25       0.08333333 0.08333333 0.33333333]
t = 2 : threshold : 35.0 : feature : 0 : loss 0.33333333333333315  : alpha = 0.3465735902799731 : sign : gt : D :  [0.1875 0.1875 0.0625 0.0625 0.5   ]


In [41]:
predict(X, best_alphas, best_indexes, best_splits, best_signs)

array([ 1.,  1., -1., -1., -1.])

In [ ]:
        indices = np.argsort( X[:,feature] );
        X_temp = X[indices]
indices

In [ ]:
scatter(X[:],np.zeros(num_egs), c = labels)

In [52]:
import sklearn 
clf = sklearn.ensemble.AdaBoostClassifier(n_estimators=1)
clf.fit(X,labels)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=1, random_state=None)

In [53]:
clf.predict(X)

array([ 1.,  1., -1., -1., -1.])

In [56]:
clf.estimators_

[DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, presort=False,
             random_state=439395978, splitter='best')]